In [ ]:
LIST @MY_AZURE_SCHEMA;


In [ ]:
CREATE OR REPLACE TABLE Open_beauty_table (
    _id VARCHAR,
    code VARCHAR,
    product_name VARCHAR,
    product_name_fr VARCHAR,
    brands VARCHAR,
    categories_tags VARCHAR,
    labels_tags VARCHAR,
    quantity VARCHAR,
    packaging VARCHAR,
    manufacturing_places VARCHAR,
    additives_tags VARCHAR,
    additives_n VARCHAR,
    allergens_tags VARCHAR,
    ingredients_text_with_allergens VARCHAR,
    ingredients_text_with_allergens_fr VARCHAR,
    ingredients_analysis_tags VARCHAR,
    periods_after_opening VARCHAR,
    generic_name VARCHAR,
    generic_name_fr VARCHAR
);


In [ ]:
COPY INTO Open_beauty_table
FROM @MY_AZURE_SCHEMA
FILE_FORMAT = (
    TYPE = CSV
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    FIELD_DELIMITER = ','  -- Délimiteur de champ pour les colonnes
    SKIP_HEADER = 1        -- Ignore la première ligne (en-têtes des colonnes)
    NULL_IF = ('')         -- Interprète les champs vides comme NULL
);


In [ ]:
select * from Open_beauty_table

In [ ]:
# Importer la session active directement depuis Snowflake
from snowflake.snowpark import Session

# Récupérer la session active
session = Session.get_active_session()

# Charger une table dans un DataFrame
df = session.table("Open_beauty_table")

# Afficher les données
df.show()


In [ ]:
pip install re

In [ ]:
# Importer les fonctions nécessaires
from snowflake.snowpark import functions as F

filtered_df = df

# Calculer le seuil pour les lignes avec un nombre acceptable de valeurs non nulles
threshold = len(filtered_df.schema.fields) / 2  # Obtenir le nombre de colonnes dans le DataFrame

# Supprimer les lignes avec trop de valeurs nulles
filtered_df = filtered_df.dropna(thresh=int(threshold))
print(f"Nombre de lignes après suppression des lignes avec trop de valeurs manquantes : {filtered_df.count()}")



# Nettoyer les colonnes avec `regexp_replace`
columns_to_clean = ['categories_tags', 'labels_tags', 'additives_tags', 'allergens_tags', 'ingredients_analysis_tags']

for column in columns_to_clean:
    filtered_df = filtered_df.with_column(
        f"{column}_cleaned",
        F.regexp_replace(F.col(column).cast("string"), r".*?:([\w-]+)", r"\1")
    )

# Supprimer les lignes contenant uniquement des "[]" ou des valeurs NULL
filtered_df = filtered_df.filter(~F.col('categories_tags_cleaned').like("[]") & F.col('categories_tags_cleaned').is_not_null())

# Supprimer les colonnes inutiles
columns_to_drop = ['additives_tags', 'additives_n', 'ingredients_text_with_allergens_fr']
filtered_df = filtered_df.drop(*columns_to_drop)

# Nettoyer la colonne `ingredients_text_with_allergens` en retirant certains mots
mots_a_supprimer = ['ingrédients', 'ingrédient', 'ingredient', 'ingredients']
pattern = '|'.join(mots_a_supprimer)




filtered_df = filtered_df.with_column(
    'ingredients_text_with_allergens',
    F.trim(F.regexp_replace(F.lower(F.col('ingredients_text_with_allergens')), pattern, ""))
)


# Filtrer les lignes où `ingredients_text_with_allergens` commence par 'Water' ou 'Aqua'
filtered_df = filtered_df.filter(
    F.col('ingredients_text_with_allergens').startswith('Water') | F.col('ingredients_text_with_allergens').startswith('Aqua')
)

# Supprimer le symbole '*' dans la colonne `ingredients_text_with_allergens`
filtered_df = filtered_df.with_column(
    'ingredients_text_with_allergens',
    F.regexp_replace(F.col('ingredients_text_with_allergens'), r"\*", "")
)

# Sauvegarder les données nettoyées dans une nouvelle table
filtered_df.write.mode("overwrite").save_as_table("cleaned_products_table")

print("Données nettoyées sauvegardées dans la table 'cleaned_products_table'.")
